# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [15]:
driver = webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/tools_search/')

In [16]:
text_input = driver.find_element_by_name("mcrdata")
text_input.send_keys('006179570C')

In [17]:
button = driver.find_element_by_id('submit3')
button.click()

# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [18]:
driver = webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/tools_search/')
text_input = driver.find_element_by_name("mcrdata")
text_input.send_keys('006179570C')
button = driver.find_element_by_id('submit3')
button.click()

# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [19]:
import pandas as pd
df = pd.read_csv("trucks-subset.csv")
df.head()

TDLR Number
0  006507931C
1  006179570C
2  006502097C

## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [20]:
url_base = ("https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=")

In [21]:
df['TDLR Number'].describe()

count     3         
unique    3         
top       006502097C
freq      1         
Name: TDLR Number, dtype: object

In [22]:
def create_url(towtruck):
    return (url_base)+(towtruck["TDLR Number"])

### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [23]:
df['create_url'] = df.apply(create_url, axis=1)
pd.set_option('display.max_colwidth', -1)
df.head()

TDLR Number  \
0  006507931C   
1  006179570C   
2  006502097C   

                                                                      create_url  
0  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C  
1  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C  
2  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C

## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [24]:
df.create_url

0    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
Name: create_url, dtype: object

## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [25]:
def open(row):
    driver.get(row['create_url'])
    
df.apply(open, axis=1)

0    None
1    None
2    None
dtype: object

In [26]:
driver = webdriver.Chrome()
#df.apply(open, axis=1)

def open(row):
    driver.get(row['create_url'])
    
    business_name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
    owner_opp = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
    phone_number = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
    license_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]').text
    address = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text.split('\n')[-3:]

    return pd.Series({
        'business_name': business_name,
        'owner_oper': owner_opp,
        'phone': phone_number,
        'license': license_status,
        'address': address        
})

 #address = driver.find_element_by_id('t1').text
 # words = []
 #   for words in address:
 #       if words==("Physical"):
 #           address.append(words)

In [27]:
new_df = df.apply(open, axis=1).join(df)
new_df.head()

business_name  \
0  Name:   AUGUSTUS E SMITH      
1  Name:   B.D. SMITH TOWING     
2  Name:   BARRY MICHAEL SMITH   

                                       owner_oper                phone  \
0  Owner/Officer:   AUGUSTUS EUGENE SMITH / OWNER  Phone:   9032276464   
1  Owner/Officer:   BRANDT SMITH / OWNER           Phone:   8173330706   
2  Owner/Officer:   BARRY MICHAEL SMITH / OWNER    Phone:   8066544404   

            license  \
0  Status:  Active    
1  Status:  Expired   
2  Status:  Active    

                                                       address TDLR Number  \
0  [Physical:, 103 N MAIN ST, BONHAM, TX. 75418]                006507931C   
1  [Physical:, 13619 BRETT JACKSON RD., FORT WORTH, TX. 76179]  006179570C   
2  [Physical:, 4501 W CEMETERY RD, CANYON, TX. 79015]           006502097C   

                                                                      create_url  
0  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C  
1  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C  
2  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C

### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [28]:
df = pd.DataFrame(new_df)

df.to_csv("tow-trucks-extended.csv", index=False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [29]:
check = pd.read_csv("tow-trucks-extended.csv")
check.head()

business_name  \
0  Name:   AUGUSTUS E SMITH      
1  Name:   B.D. SMITH TOWING     
2  Name:   BARRY MICHAEL SMITH   

                                       owner_oper                phone  \
0  Owner/Officer:   AUGUSTUS EUGENE SMITH / OWNER  Phone:   9032276464   
1  Owner/Officer:   BRANDT SMITH / OWNER           Phone:   8173330706   
2  Owner/Officer:   BARRY MICHAEL SMITH / OWNER    Phone:   8066544404   

            license  \
0  Status:  Active    
1  Status:  Expired   
2  Status:  Active    

                                                             address  \
0  ['Physical:', '103 N MAIN ST', 'BONHAM, TX. 75418']                 
1  ['Physical:', '13619 BRETT JACKSON RD.', 'FORT WORTH, TX. 76179']   
2  ['Physical:', '4501 W CEMETERY RD', 'CANYON, TX. 79015']            

  TDLR Number  \
0  006507931C   
1  006179570C   
2  006502097C   

                                                                      create_url  
0  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C  
1  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C  
2  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C

## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [32]:
alltrucks = pd.read_csv("tow-trucks.csv")
alltrucks 

TDLR Number
0   006507931C
1   006179570C
2   006502097C
3   006494912C
4   0649468VSF
5   006448786C
6   0648444VSF
7   0651667VSF
8   006017767C
9   006495492C
10  006518521C
11  006521662C
12  006482451C
13  0646264VSF
14  0650690VSF
15  006505010C
16  0000210VSF
17  005029369C
18  006410509C
19  0643482VSF

In [36]:
def all_url(alltrucks):
    return (url_base)+(alltrucks["TDLR Number"])

In [44]:
alltrucks['all_url'] = alltrucks.apply(all_url, axis=1)
pd.set_option('display.max_colwidth', -1)
alltrucks.head(20)

TDLR Number  \
0   006507931C   
1   006179570C   
2   006502097C   
3   006494912C   
4   0649468VSF   
5   006448786C   
6   0648444VSF   
7   0651667VSF   
8   006017767C   
9   006495492C   
10  006518521C   
11  006521662C   
12  006482451C   
13  0646264VSF   
14  0650690VSF   
15  006505010C   
16  0000210VSF   
17  005029369C   
18  006410509C   
19  0643482VSF   

                                                                          all_url  
0   https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C  
1   https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C  
2   https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C  
3   https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C  
4   https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF  
5   https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006448786C  
6   https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0648444VSF  
7   https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0651667VSF  
8   https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006017767C  
9   https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C  
10  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006518521C  
11  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006521662C  
12  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006482451C  
13  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0646264VSF  
14  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0650690VSF  
15  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006505010C  
16  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0000210VSF  
17  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=005029369C  
18  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006410509C  
19  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0643482VSF

In [45]:
driver = webdriver.Chrome()
alltrucks.apply(open, axis=1)

def open(row):
    driver.get(row['all_url'])
    
    business_name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
    owner_opp = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
    phone_number = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
    license_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]').text
    address = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text.split('\n')[-3:]

    return pd.Series({
        'business_name': business_name,
        'owner_oper': owner_opp,
        'phone': phone_number,
        'license': license_status,
        'address': address        
})

In [46]:
new_df = alltrucks.apply(open, axis=1).join(alltrucks)
new_df.head()

business_name  \
0  Name:   AUGUSTUS E SMITH      
1  Name:   B.D. SMITH TOWING     
2  Name:   BARRY MICHAEL SMITH   
3  Name:   HEATH SMITH           
4  Name:   HEATH SMITH           

                                       owner_oper                  phone  \
0  Owner/Officer:   AUGUSTUS EUGENE SMITH / OWNER  Phone:   9032276464     
1  Owner/Officer:   BRANDT SMITH / OWNER           Phone:   8173330706     
2  Owner/Officer:   BARRY MICHAEL SMITH / OWNER    Phone:   8066544404     
3  Owner/Officer:   HEATH A SMITH / OWNER          Phone:   940-552-0687   
4  Owner/Officer:   HEATH A SMITH / OWNER          Phone:   9405520687     

            license  \
0  Status:  Active    
1  Status:  Expired   
2  Status:  Active    
3  Status:  Expired   
4  Status:  Expired   

                                                       address TDLR Number  \
0  [Physical:, 103 N MAIN ST, BONHAM, TX. 75418]                006507931C   
1  [Physical:, 13619 BRETT JACKSON RD., FORT WORTH, TX. 76179]  006179570C   
2  [Physical:, 4501 W CEMETERY RD, CANYON, TX. 79015]           006502097C   
3  [Physical:, 1529 WILBARGER ST, VERNON, TX. 76384]            006494912C   
4  [Physical:, 1529 WILBARGER ST, VERNON, TX. 76384]            0649468VSF   

                                                                         all_url  
0  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C  
1  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C  
2  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C  
3  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C  
4  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF

In [47]:
df = pd.DataFrame(new_df)

df.to_csv("tow-trucks-extended.csv", index=False)

In [50]:
check = pd.read_csv("tow-trucks-extended.csv")
check.head()

business_name  \
0  Name:   AUGUSTUS E SMITH      
1  Name:   B.D. SMITH TOWING     
2  Name:   BARRY MICHAEL SMITH   
3  Name:   HEATH SMITH           
4  Name:   HEATH SMITH           

                                       owner_oper                  phone  \
0  Owner/Officer:   AUGUSTUS EUGENE SMITH / OWNER  Phone:   9032276464     
1  Owner/Officer:   BRANDT SMITH / OWNER           Phone:   8173330706     
2  Owner/Officer:   BARRY MICHAEL SMITH / OWNER    Phone:   8066544404     
3  Owner/Officer:   HEATH A SMITH / OWNER          Phone:   940-552-0687   
4  Owner/Officer:   HEATH A SMITH / OWNER          Phone:   9405520687     

            license  \
0  Status:  Active    
1  Status:  Expired   
2  Status:  Active    
3  Status:  Expired   
4  Status:  Expired   

                                                             address  \
0  ['Physical:', '103 N MAIN ST', 'BONHAM, TX. 75418']                 
1  ['Physical:', '13619 BRETT JACKSON RD.', 'FORT WORTH, TX. 76179']   
2  ['Physical:', '4501 W CEMETERY RD', 'CANYON, TX. 79015']            
3  ['Physical:', '1529 WILBARGER ST', 'VERNON, TX. 76384']             
4  ['Physical:', '1529 WILBARGER ST', 'VERNON, TX. 76384']             

  TDLR Number  \
0  006507931C   
1  006179570C   
2  006502097C   
3  006494912C   
4  0649468VSF   

                                                                         all_url  
0  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C  
1  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C  
2  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C  
3  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C  
4  https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF